In [27]:
!pip install -q langextract qdrant-client sentence-transformers beautifulsoup4 lxml requests openai google-generativeai fastembed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.8 MB/s eta 0:00:00


# **Mount Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Imports**

In [39]:
import os
import re
import uuid
from lxml import etree
from openai import OpenAI
from qdrant_client.models import NamedVector
import langextract as lx
from langextract.inference import OpenAILanguageModel
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct, Filter, FieldCondition, MatchValue
import textwrap
from sentence_transformers import SentenceTransformer
from fastembed.rerank.cross_encoder import TextCrossEncoder
import nltk

In [4]:

os.environ["OPENAI_API_KEY"] = "sk-..."

In [5]:
print("LangExtract OK")
print("Qdrant local mode OK ->", QdrantClient(":memory:"))
print("ST dimension ->", SentenceTransformer("intfloat/e5-base-v2").get_sentence_embedding_dimension())


LangExtract OK
Qdrant local mode OK -> <qdrant_client.qdrant_client.QdrantClient object at 0x7e12cdf91390>


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

ST dimension -> 768


# **Prompt**

In [6]:
prompt_legal = textwrap.dedent("""\
Extract structured legal facts from the case text. Classes:

1) CaseName {name}
2) Date {date, type}  # e.g., judgment date, hearing date
3) Citation {citation}
4) Court {name}
5) Judge {name}
6) Party {role, name}
7) Catchphrase {phrase}
8) Legislation {act, section}
9) Order {order_text}
10) Penalty {amount, recipient}
11) Keyword {keyword}
12) ProceduralHistory {description}
13) BusinessEntity {name, role}
14) DeceptionMethod {description}

Rules:
- Use exact text spans from the original.
- Omit any field if not explicitly stated.
- Preserve original casing and punctuation.
- Return each occurrence as a separate extraction.
""")


# **Few-shot examples**

In [7]:
examples_legal = [
    lx.data.ExampleData(
        text=(
            "Federal Court of Australia\n"
            "Smith v Jones [2020] FCA 123 (5 March 2020)\n"
            "Before: Justice Brown\n"
            "Catchwords: contract law, misrepresentation\n"
            "Orders: The appeal is dismissed.\n"
            "Under the Bankruptcy Act 1966 (Cth), section 35A, "
            "the proceedings are transferred to the Family Court.\n"
            "Penalty: Smith to pay $50,000 to Jones.\n"
            "History: Appeal from the decision of the District Court of New South Wales."
        ),
        extractions=[
            lx.data.Extraction(
                extraction_class="CaseName",
                extraction_text="Smith v Jones",
                attributes={"name": "Smith v Jones"}
            ),
            lx.data.Extraction(
                extraction_class="Date",
                extraction_text="5 March 2020",
                attributes={"date": "5 March 2020", "type": "judgment"}
            ),
            lx.data.Extraction(
                extraction_class="Citation",
                extraction_text="[2020] FCA 123",
                attributes={"citation": "[2020] FCA 123"}
            ),
            lx.data.Extraction(
                extraction_class="Court",
                extraction_text="Federal Court of Australia",
                attributes={"name": "Federal Court of Australia"}
            ),
            lx.data.Extraction(
                extraction_class="Judge",
                extraction_text="Justice Brown",
                attributes={"name": "Justice Brown"}
            ),
            lx.data.Extraction(
                extraction_class="Keyword",
                extraction_text="contract law",
                attributes={"keyword": "contract law"}
            ),
            lx.data.Extraction(
                extraction_class="Keyword",
                extraction_text="misrepresentation",
                attributes={"keyword": "misrepresentation"}
            ),
            lx.data.Extraction(
                extraction_class="Order",
                extraction_text="The appeal is dismissed.",
                attributes={"order_text": "The appeal is dismissed."}
            ),
            lx.data.Extraction(
                extraction_class="Legislation",
                extraction_text="Bankruptcy Act 1966 (Cth), section 35A",
                attributes={"act": "Bankruptcy Act 1966 (Cth)", "section": "section 35A"}
            ),
            lx.data.Extraction(
                extraction_class="Penalty",
                extraction_text="$50,000",
                attributes={"amount": "$50,000", "recipient": "Jones"}
            ),
            lx.data.Extraction(
                extraction_class="ProceduralHistory",
                extraction_text="Appeal from the decision of the District Court of New South Wales",
                attributes={"description": "Appeal from the decision of the District Court of New South Wales"}
            )
        ]
    )
]


# **Parse XML and Clean Text**

In [8]:
import re
from lxml import etree

def get_text_from_xml(filepath):
    """
    Extract and clean text from XML by recursively collecting text nodes
    and removing inline XML artifacts.
    """
    try:
        parser = etree.XMLParser(recover=True)
        tree = etree.parse(filepath, parser)
        root = tree.getroot()

        def recursive_text(elem):
            texts = []
            if elem.text and elem.text.strip():
                clean_text = re.sub(r'"id=c\d+">', '', elem.text)
                texts.append(clean_text.strip())
            for child in elem:
                texts.extend(recursive_text(child))
            if elem.tail and elem.tail.strip():
                clean_tail = re.sub(r'"id=c\d+">', '', elem.tail)
                texts.append(clean_tail.strip())
            return texts

        all_text = recursive_text(root)
        filtered_text = [line for line in all_text if len(line) > 50]

        joined_text = "\n".join(filtered_text)
        return joined_text

    except Exception as e:
        print(f"Error parsing {filepath}: {e}")
        return ""


In [9]:
def remove_urls(text):
    return re.sub(r'http[s]?://\S+', '', text)

# **Setup Qdrant and Embedder**

In [18]:

FULLTEXT_DIR = "/content/drive/MyDrive/legal_cases/fulltext"
COLLECTION_NAME = "legal_facts"

qdrant = QdrantClient(path="/content/drive/MyDrive/qdrant_data")
embedder = SentenceTransformer("intfloat/e5-base-v2")
dim = embedder.get_sentence_embedding_dimension()

qdrant.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=dim, distance=Distance.COSINE),
)

/tmp/ipython-input-3039892705.py:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

# **Helper for character offsets**

In [19]:
def get_offsets(ex):
    ci = getattr(ex, "char_interval", None)
    return (getattr(ci, "start_pos", None), getattr(ci, "end_pos", None)) if ci else (None, None)

# **LangExtract extraction with Qdrant indexing**

In [20]:
max_files = 50
files_to_process = sorted([f for f in os.listdir(FULLTEXT_DIR) if f.lower().endswith(".xml")])[:max_files]

total_points = 0

for filename in files_to_process:
    filepath = os.path.join(FULLTEXT_DIR, filename)
    sample_text = remove_urls(get_text_from_xml(filepath))
    if not sample_text.strip():
        continue
    try:
        result = lx.extract(
            text_or_documents=sample_text,
            prompt_description=prompt_legal,
            examples=examples_legal,
            language_model_type=OpenAILanguageModel,
            model_id="gpt-4o-mini",
            api_key=os.environ["OPENAI_API_KEY"],
            fence_output=True,
            use_schema_constraints=True,
            extraction_passes=1,
            max_workers=2,
            max_char_buffer=2000
        )
    except Exception as e:
        print(f"Skipping {filename} due to extraction error: {e}")
        continue

    evidence_texts, metas = [], []
    for ex in result.extractions:
        if not getattr(ex, "attributes", None):
            continue
        start, end = get_offsets(ex)
        if not (isinstance(start, int) and isinstance(end, int) and end > start):
            continue
        snippet = sample_text[start:end].strip()
        if not snippet:
            continue
        evidence_texts.append(snippet)
        metas.append({
            "entity_type": ex.extraction_class,
            "subject": ex.extraction_text,
            "attributes": ex.attributes,
            "doc_id": os.path.splitext(filename)[0],
            "start_char": start,
            "end_char": end,
            "evidence_text": snippet
        })

    if evidence_texts:
        vectors = embedder.encode(evidence_texts, batch_size=32).tolist()
        points = []
        for vec, meta in zip(vectors, metas):
            fact_id = str(uuid.uuid4())
            payload = dict(meta)
            payload["fact_id"] = fact_id
            points.append(PointStruct(id=fact_id, vector=vec, payload=payload))
        qdrant.upsert(collection_name=COLLECTION_NAME, points=points)
        total_points += len(points)
        print(f"Indexed {len(points)} facts from {filename}")

print(f"\nTotal facts indexed into Qdrant: {total_points}")


/tmp/ipython-input-859150529.py:12: DeprecationWarning: 'language_model_type' is deprecated and will be removed in v2.0.0. Use model, config, or model_id parameters instead.
  result = lx.extract(
DEBUG:absl:Registered GeminiLanguageModel with patterns ['^gemini'] at priority 10
DEBUG:absl:Registered OllamaLanguageModel with patterns ['^gemma', '^llama', '^mistral', '^mixtral', '^phi', '^qwen', '^deepseek', '^command-r', '^starcoder', '^codellama', '^codegemma', '^tinyllama', '^wizardcoder', '^gpt-oss', '^meta-llama/[Ll]lama', '^google/gemma', '^mistralai/[Mm]istral', '^mistralai/[Mm]ixtral', '^microsoft/phi', '^Qwen/', '^deepseek-ai/', '^bigcode/starcoder', '^codellama/', '^TinyLlama/', '^WizardLM/'] at priority 10
DEBUG:absl:Registered OpenAILanguageModel with patterns ['^gpt-4', '^gpt4\\.', '^gpt-5', '^gpt5\\.'] at priority 10
2025-08-16 04:28:03,921 - langextract.debug - DEBUG - [langextract.inference] CALL: BaseLanguageModel.__init__(self=<OpenAILanguageModel>, constraint=Constrai

✓ Extraction processing complete



INFO:absl:Finalizing annotation for document ID doc_ee7ba04f.
INFO:absl:Document annotation completed.


✓ Extracted 3 entities (3 unique types)
  • Time: 4.16s
  • Speed: 24 chars/sec
  • Chunks: 1
Indexed 3 facts from 06_1048.xml


2025-08-16 04:28:09,074 - langextract.debug - DEBUG - [langextract.inference] CALL: BaseLanguageModel.__init__(self=<OpenAILanguageModel>, constraint=Constraint(co...NONE: 'none'>), kwargs={})
2025-08-16 04:28:09,075 - langextract.debug - DEBUG - [langextract.inference] RETURN: BaseLanguageModel.__init__ -> None (0.0 ms)
2025-08-16 04:28:09,077 - langextract.debug - DEBUG - [langextract.inference] CALL: BaseLanguageModel.apply_schema(self=<OpenAILanguageModel>, schema_instance=None)
2025-08-16 04:28:09,079 - langextract.debug - DEBUG - [langextract.inference] RETURN: BaseLanguageModel.apply_schema -> None (0.0 ms)
DEBUG:absl:Initialized Annotator with prompt:
Extract structured legal facts from the case text. Classes:

1) CaseName {name}
2) Date {date, type}  # e.g., judgment date, hearing date
3) Citation {citation}
4) Court {name}
5) Judge {name}
6) Party {role, name}
7) Catchphrase {phrase}
8) Legislation {act, section}
9) Order {order_text}
10) Penalty {amount, recipient}
11) Keywo

...

Indexed 3 facts from 06_1472.xml


LangExtract: model=gpt-4o-mini [00:00]2025-08-16 04:32:27,642 - langextract.debug - DEBUG - [langextract.tokenizer] CALL: tokenize(text='Paligorov v Cohen [2006] FCA 1473 (10 November 2006)\n\nappeal from sequestration order of federal magistrate\nconstruction and effect of undertaking given by appellant in form of consent orders')
2025-08-16 04:32:27,643 - langextract.debug - DEBUG - [langextract.tokenizer] RETURN: tokenize -> TokenizedText...wline=False)]) (0.3 ms)
INFO:absl:Processing batch 0 with length 1
DEBUG:absl:Token util returns string: Paligorov v Cohen [2006] FCA 1473 (10 November 2006)

appeal from sequestration order of federal magistrate
construction and effect of undertaking given by appellant in form of consent orders for tokenized_text: TokenizedText(text='Paligorov v Cohen [2006] FCA 1473 (10 November 2006)\n\nappeal from sequestration order of federal magistrate\nconstruction and effect of undertaking given by appellant in form of consent orders', tokens=[Token(inde

✓ Extraction processing complete



INFO:absl:Finalizing annotation for document ID doc_50257052.
INFO:absl:Document annotation completed.


✓ Extracted 5 entities (5 unique types)
  • Time: 4.77s
  • Speed: 40 chars/sec
  • Chunks: 1
Indexed 5 facts from 06_1473.xml

Total facts indexed into Qdrant: 188


# **Retrieve and re-rank**

In [36]:
reranker = TextCrossEncoder(model_name="jinaai/jina-reranker-v2-base-multilingual")


In [37]:
nltk.download("punkt")

def get_full_context(doc_id, start_char, end_char, window=100):
    if doc_id not in source_docs:
        return None
    text = source_docs[doc_id]
    start = max(0, start_char - window)
    end = min(len(text), end_char + window)
    snippet = text[start:end].strip()
    sentences = nltk.sent_tokenize(snippet)
    return " ".join(sentences)


def rerank_results(query, results):
    rerank_input = [(query, r["evidence_text"]) for r in results]
    scores = reranker.rerank(query, [r["evidence_text"] for r in results])
    ranked = sorted(
        zip(results, scores),
        key=lambda x: x[1],
        reverse=True
    )
    return [
        {**res, "rerank_score": score}
        for res, score in ranked
    ]

def search_semantic(query, entity_type=None, limit=10):
    qvec = embedder.encode(query).tolist()
    qfilter = None
    if entity_type:
        qfilter = Filter(must=[FieldCondition(key="entity_type", match=MatchValue(value=entity_type))])

    hits = qdrant.query_points(
        collection_name=COLLECTION_NAME,
        query=qvec,
        query_filter=qfilter,
        limit=limit,
        with_payload=True
    )

    results = []
    for p in hits.points:
        subj = p.payload["subject"]
        evidence = p.payload["evidence_text"]

        # Always expand context
        ctx = get_full_context(
            p.payload["doc_id"],
            p.payload["start_char"],
            p.payload["end_char"],
            window=200  # increase window size if you want more text
        )
        if ctx:
            evidence = ctx

        results.append({
            "entity_type": p.payload["entity_type"],
            "subject": subj,
            "evidence_text": evidence,
            "score": float(p.score)
        })

    # === Rerank ===
    new_scores = list(reranker.rerank(query, [r["evidence_text"] for r in results]))
    for r, s in zip(results, new_scores):
        r["rerank_score"] = s

    results.sort(key=lambda x: x["rerank_score"], reverse=True)
    return results


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Wrap results into a context string**

In [38]:
def format_results_for_llm(results, top_k=5):
    formatted = []
    for i, r in enumerate(results[:top_k], start=1):
        formatted.append(
            f"Result {i}:\n"
            f"- Entity Type: {r['entity_type']}\n"
            f"- Subject: {r['subject']}\n"
            f"- Evidence: {r['evidence_text']}\n"
        )
    return "\n\n".join(formatted)


# **Ask the LLM to produc clean response**

In [44]:

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

def generate_clean_answer(query, results):
    context = format_results_for_llm(results, top_k=5)
    prompt = f"""
You are a legal research assistant writing a professional case law summary.
A user has asked: "{query}".

Based on the retrieved legal evidence, write a clear, professional answer in plain English.
- Focus only on the relevant facts and legal principles.
- Do not mention the retrieval process, evidence chunks, or that you are summarizing documents.
- Do not say things like "your query does not directly match".
- Write as if you are drafting a concise legal brief, suitable for a lawyer or law student.
- Keep the answer factual and objective.
- If the evidence is indirect, frame it as "Relevant cases indicate..." instead of disclaiming.

Evidence:
{context}
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content



# **Example search**

In [45]:
results = search_semantic("application under section 79 of the Family Law Act")
answer = generate_clean_answer("application under section 79 of the Family Law Act", results)
print(answer)


An application under Section 79 of the Family Law Act pertains to the division of property and financial resources following the breakdown of a marriage or de facto relationship. This section empowers the Family Court to make orders regarding the distribution of property between parties, ensuring a just and equitable outcome.

Key legal principles include:

1. **Jurisdiction**: The Family Court has the authority to determine property disputes under Section 79, which applies to both married and de facto couples.

2. **Considerations for Orders**: The court considers various factors when making property settlement orders, including the financial and non-financial contributions of each party, the future needs of the parties, and the overall circumstances of the relationship.

3. **Equitable Distribution**: The aim is to achieve a fair distribution of assets, which may not necessarily mean an equal split. The court assesses the contributions made by each party and their respective needs po